<a href="https://colab.research.google.com/github/AlexandrBobrov/Case_Flocktory/blob/main/finetune_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
file_name = "talks_dataset.df"

In [3]:
load_df = pd.read_pickle(file_name)
load_df.sample(3)

FileNotFoundError: [Errno 2] No such file or directory: 'talks_dataset.df'

In [ ]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install bitsandbytes einops wandb -Uqqq
#!pip install datasets -q

In [ ]:
import torch
import glob
import pandas as pd
import numpy as np
import re
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
from datasets import Dataset
import warnings
import random
warnings.filterwarnings("ignore")

In [ ]:
trained_model_dir = 'tinyllama/checkpoint-700'

config = PeftConfig.from_pretrained(trained_model_dir)

In [ ]:
# Loading the model with double quantization
model_name = "PY007/TinyLlama-1.1B-step-50K-105b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)


trained_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map='auto'

    )


In [ ]:
# Creating tokenizer and defining the pad token
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, padding_side='right')
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Generating lyrics with the base model. The repetition penalty in the generation config prevents the model from continually repeating the same string.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def generate_answer(query, model):
    encoding = tokenizer(query, return_tensors="pt").to(device)
    generation_config = GenerationConfig(max_new_tokens=250, pad_token_id = tokenizer.eos_token_id,repetition_penalty=1.3, eos_token_id = tokenizer.eos_token_id)
    outputs = model.generate(input_ids=encoding.input_ids, generation_config=generation_config)
    text_output = tokenizer.decode(outputs[0],skip_special_tokens=True)
    return text_output.split("[/INST]")[1].split("\n\n")[0]

def get_test_prompt(df):
    use_col = ['speaker', 'query', 'context', "companions", "close_reply"]
    j = random.randint(0, df.shape[0])
    id = random.randint(0, len(df[use_col].iloc[j]["companions"]))
    query = df[use_col].iloc[j]['query']
    role = df[use_col].iloc[j]['companions'][id]
    context = df[use_col].iloc[j]['context']
    reply = df.iloc[j][use_col]["close_reply"][id]
    prompt = f"[INST]"
    prompt += f'Use the given context to guide your an about the query like indicated in your role'
    prompt += f"query: {query}\n\n"
    prompt += f"context: {context}\n\n"
    prompt += f"your role: {role}\n\n"
    prompt += f'answer:[/INST]'

    return query, role, reply, prompt

In [ ]:
query, role, reply, prompt = get_test_prompt(load_df)
answer = generate_answer(prompt, trained_model)

print(f"\n\nQuery: {query}\n")
print(f"Trained_model reply with role: {role}\n")

print(answer)
print()
print(f'Answer in df: {reply}')